## Parafac

Now the CANDECOMP/PARAFAC decomposition. The above tensor is too high a rank to reasonably decompose in this example, so we instead generate an example sparse tensor from a random sparse factorization and re-factor it.

In [ ]:
import numpy as np
import sparse
import tensorly as tl

In [2]:
shape = (1000, 1001, 1002, 100)
rank = 5

starting_factors = [sparse.random((i, rank)) for i in shape]
starting_factors
starting_weights = sparse.ones(rank)

Now convert it to a tensor. It is very important to use `kruskal_to_tensor` from the sparse backend, as a fully dense version of the tensor would use several TB of memory.

In [3]:
from tensorly.contrib.sparse.kruskal_tensor import kruskal_to_tensor
tensor = kruskal_to_tensor((starting_weights, starting_factors))
tensor

Format,coo
Data Type,float64
Shape,"(1000, 1001, 1002, 100)"
nnz,5258
Density,5.242262727292668e-08
Read-only,True
Size,205.4K
Storage ratio,0.0


As before, we can compare the actual spase used by the tensor vs. what it would require if it were dense.

In [4]:
tensor.nbytes / 1e9                # Actual memory usage in GB

0.00021032

In [5]:
np.prod(tensor.shape) * 8 / 1e9    # Memory usage if array was dense, in GB

802.4016

In [6]:
import time
%load_ext memory_profiler

In [7]:
from tensorly.decomposition import parafac

Now we can decompose the tensor. Note again how much memory is actually used.

In [8]:
%%memit
start_time = time.time()
factors = parafac(tensor, rank=rank, init='random', verbose=True)
end_time = time.time()
total_time = end_time - start_time
print('Took %d mins %d secs' % (divmod(total_time, 60)))

RuntimeError: Cannot convert a sparse array to dense automatically. To manually densify, use the todense method.

In [9]:
type(factors[0])

numpy.ndarray

In [10]:
[i.shape for i in factors]

[(1000, 5), (1001, 5), (1002, 5), (100, 5)]

Note that even though we started with a sparse tensor, the factors are dense. This is because we used the dense version of `parafac`. Since the factors are in general dense, even for a sparse tensor, this is generally preferred. 

We can also use the dense version of `parafac`. It should give the same answer, though it may be slower.

In [9]:
from tensorly.contrib.sparse.decomposition import parafac as parafac_sparse

In [10]:
%%memit
start_time = time.time()
sparse_kruskal = parafac_sparse(tensor, rank=rank, init='random', verbose=True)
end_time = time.time()
total_time = end_time - start_time
print('Took %d mins %d secs' % (divmod(total_time, 60)))

reconstruction error=0.47643618036275764
iteration 1, reconstruction error: 0.24060082750067524, decrease = 0.2358353528620824, unnormalized = 2.788771147576186
iteration 2, reconstruction error: 0.21178785204968922, decrease = 0.028812975450986017, unnormalized = 2.4548039062818674
iteration 3, reconstruction error: 0.21000039557804184, decrease = 0.0017874564716473806, unnormalized = 2.4340857438072825
iteration 4, reconstruction error: 0.2089596003743366, decrease = 0.0010407952037052304, unnormalized = 2.422022029543371
iteration 5, reconstruction error: 0.20823371325758314, decrease = 0.0007258871167534764, unnormalized = 2.4136083716660153
iteration 6, reconstruction error: 0.20770079593574342, decrease = 0.0005329173218397154, unnormalized = 2.4074314001791404
iteration 7, reconstruction error: 0.20729671601288788, decrease = 0.0004040799228555436, unnormalized = 2.4027477652894342
iteration 8, reconstruction error: 0.20698307867445712, decrease = 0.00031363733843076114, unnorma

Let's look at the result

In [11]:
sparse_kruskal

(weights, factors) : rank-5 KruskalTensor of shape (1000, 1001, 1002, 100) 

Because the `factors_sparse` are sparse, we can reconstruct them into a tensor without using too much memory. In general, this will not be the case, but it is for our toy example. Let's do this to look at the absolute error for the decomposition. 

In [12]:
tl.norm(tensor - kruskal_to_tensor(sparse_kruskal))

2.3815669628500578

It is not actually necessary to compute this, as the same as the norm of the tensor times the reconstruction error that was printed by the algorithm (you can pass `return_errors=True` to `parafac()` to have the reconstruction errors be returned along with the factors). That is, $$\mathrm{reconstruction\ error} = \frac{\|\mathrm{tensor} - \mathrm{kruskal\_to\_tensor}(\mathrm{factors})\|_2}{\|\mathrm{tensor}\|_2}$$ (they won't be exactly the same due to numerical differences in how they are calculated).

In [13]:
tl.norm(tensor - kruskal_to_tensor(sparse_kruskal))/tl.norm(tensor)

0.20546934534517586

Let's look at one of the nonzero entries to see how close it is to the original tensor. The factors satisfy $$\sum_{r=0}^{R-1} {f_0}_r\circ {f_1}_r \circ {f_2}_r \circ {f_3}_r,$$ where $R$ is the rank (here 5), ${f_i}_r$ is the $r$-th column of the $i$-th factor of the decomposition, and $\circ$ is the vector outer product. Component-wise, this translates to a product of corresponding elements per component for each factor, summed over the columns.

In [14]:
tensor.coords

array([[103, 103, 103, ..., 930, 930, 930],
       [204, 204, 204, ..., 905, 905, 905],
       [ 77,  77,  77, ..., 963, 963, 963],
       [ 14,  29,  56, ...,  14,  29,  56]])

In [15]:
orig_val = tensor[tuple(tensor.coords.T[0])]
orig_val

0.016216060526249777

In [16]:
dense_val = np.sum(np.prod(np.stack([factors[i][idx] for i, idx in enumerate(tuple(tensor.coords.T[0]))], 0), 0))
dense_val

NameError: name 'factors' is not defined

And the same for the sparse factors

In [ ]:
weights_sparse, factors_sparse = sparse_kruskal
sparse_val = np.sum(np.prod(sparse.stack([factors_sparse[i][idx] for i, idx in enumerate(tuple(tensor.coords.T[0]))], 0), 0))
sparse_val

In [ ]:
np.abs(orig_val - dense_val)

In [ ]:
np.abs(orig_val - sparse_val)

The difference here is mostly due to random chance. The total reconstruction errors for the two runs of algorithm were roughly the same. In general, the error of the factorization will vary due to the randomness of the initial factors chosen by the algorithm.